#### Imports

In [2775]:
import pandas as pd
import numpy as np
import re

#### Read linkedin csv

In [2776]:
linkedin = pd.read_csv('../data/linkedin.csv')
linkedin.sample(3)

,job_id,job_title,company_name,company_state,salary_range,remote_ratio,employment_type,experience_level,company_size,image_link
682,3773802052,Data Analyst,Promise,"· Fairfield, CA",70 $/h - 80 $/h,Presencial,Contrato por obra,Intermedio,De 11 a 50 empleados · Desarrollo de software,https://media.licdn.com/dms/image/C560BAQH8jYM...
623,3772209411,Data Engineer with Power BI,InvestM Technology LLC,"· Nueva York, NY",55 $/h - 58 $/h,Presencial,Contrato por obra,Sin experiencia,De 11 a 50 empleados · Servicios y consultoría...,https://media.licdn.com/dms/image/D4E0BAQHkkQT...
376,3765481174,Data Analyst,The Society of St. Vincent de Paul - Phoenix,"· Phoenix, AZ",55 $/h - 60 $/h,Presencial,Contrato por obra,Algo de responsabilidad,De 51 a 200 empleados · Gestión de organizacio...,https://media.licdn.com/dms/image/C560BAQEb9oR...


#### Cleaning 'company_state' column

In [2777]:
states = ['AL', 'AK', 'AZ', 'AR', 'CA', 'NC', 'SC', 'CO', 'CT', 'ND', 'SD', 'DE', 'FL', 'GA', 'HI', 'ID', 'IL', 'IN', 'IA', 'KS', 'KY', 'LA',
          'ME', 'MD', 'MA', 'MI', 'MN', 'MS', 'MO', 'MT', 'NE', 'NV', 'NJ', 'NY', 'NH', 'NM', 'OH', 'OK', 'OR', 'PA', 'RI', 'TN', 'TX', 'UT',
          'UT', 'VT', 'VA', 'WV', 'WA', 'WI', 'WY']
linkedin['company_state'].value_counts()

company_state
· Estados Unidos       102
· Nueva York, NY        42
· Seattle, WA           29
· San Francisco, CA     26
· Chicago, IL           18
                      ... 
· Waukegan, IL           1
· Foxborough, MA         1
· Newport News, VA       1
· Chantilly, VA          1
· Mesa, AZ               1
Name: count, Length: 315, dtype: int64

In [2778]:
def fclean_company_state(df, states):
    for i in states:
        linkedin['company_state'] = linkedin['company_state'].apply(lambda row: i if i in str(row) else row)
    return df

linkedin = fclean_company_state(linkedin, states)

In [2779]:
def sclean_company_state(df, states):
    for key, value in states.items():
        linkedin['company_state'] = linkedin['company_state'].apply(lambda row: value if key in str(row) else row)
    return df

left_states = {
    'Virginia': 'VA',
    'Minnesota': 'MN',
    'Washington': 'WA',
    'San Francisco': 'CA',
    'Nebraska': 'NE',
    'California': 'CA',
    'Texas': 'TX',
    'Nueva York': 'NY',
    'Michigan': 'MI',
    'Luisiana': 'LA',
    'Florida': 'FL',
    'Maryland': 'MD',
    'Nevada': 'NV',
    'Oregón': 'OR',
    'Oklahoma': 'OK',
    'Georgia': 'GA',
    'Carolina del Sur': 'SC',
    'Delaware': 'DE',
    'Ohio': 'OH',
    'Arkansas': 'AR',
    'Iowa': 'IA',
    'Carolina del Norte': 'NC',
    'Indiana': 'IN',
    'Nashville': 'TN',
    'Massachusetts': 'MA',
    'Tennessee': 'TN',
    'Enid': 'OK',
    'Chicago': 'IL',
    'América del Norte': 'USA',
    'Estados Unidos': 'USA',
    'Des Moines': 'IA',
    'Pittsburgh': 'PA',
    'Cincinnati': 'OH',
    'St. Louis': 'MO',
    'Illinois': 'IL',
    'Jacksonville': 'FL'
}
linkedin = sclean_company_state(linkedin, left_states)
linkedin = linkedin[~linkedin['company_state'].str.contains('Unión Europea', na=False)]
linkedin['company_state'].unique()

array(['CA', 'NC', 'VA', 'NY', 'USA', 'MI', 'WA', 'NE', 'TX', 'IN', 'GA',
       'AZ', 'UT', 'OR', 'IL', 'MN', 'MA', 'NJ', 'PA',
       '· Cleveland y alrededores', 'OH', 'FL', 'KS', 'SC', 'MD', 'WI',
       '· Los Ángeles y alrededores', 'AR', 'MO', 'CO', 'CT', 'TN', 'KY',
       '· Urbana-Champaign y alrededores', 'LA', 'NV', 'OK', 'IA', 'SD',
       'VT', 'ID', 'AL', 'HI', 'DE', 'RI', 'NM', 'NH',
       '· Houston y alrededores'], dtype=object)

#### Cleaning 'remote_ratio' column

In [2780]:
def clean_remote_ratio(df, list_):
    for i in list_:
        df['remote_ratio'] = df['remote_ratio'].apply(lambda row: i if i in str(row) else str(row))
    df['remote_ratio'] = df['remote_ratio'].apply(lambda row: np.nan if str(row) == 'nan' else str(row))
    return df

remote_dict = [
    'Presencial',
    'Híbrido',
    'En remoto'
]
linkedin = clean_remote_ratio(linkedin, remote_dict)
linkedin['remote_ratio'].value_counts()

remote_ratio
En remoto     353
Híbrido       275
Presencial    267
Name: count, dtype: int64

In [2781]:
linkedin.loc[linkedin['remote_ratio'] == 'nan', 'remote_ratio'] = np.nan
linkedin['remote_ratio'].value_counts()

remote_ratio
En remoto     353
Híbrido       275
Presencial    267
Name: count, dtype: int64

#### Cleaning 'experience_level' column

In [2782]:
linkedin.sample(5)
linkedin['experience_level'].value_counts()

experience_level
Intermedio                 398
Sin experiencia            298
Prácticas                  116
Algo de responsabilidad     83
Director                    17
Ejecutivo                    2
Name: count, dtype: int64

#### Cleaning 'employment_type' column

In [2783]:
def clean_employment_type(df, list_):
    for i in list_:
        df['employment_type'] = df['employment_type'].apply(lambda row: i if i in str(row) else str(row))
    return df

remote_dict = [
    'Jornada completa',
    'Contrato por obra',
    'Media jornada',
    'Prácticas',
    'Temporal'
]
linkedin = clean_employment_type(linkedin, remote_dict)
linkedin['employment_type'].value_counts()

employment_type
Jornada completa     645
Contrato por obra    132
Prácticas             89
Media jornada         52
Temporal               1
Name: count, dtype: int64

#### Cleaning 'company_size' column

In [2784]:
linkedin['company_size'].value_counts()

company_size
De 51 a 200 empleados · Servicios y consultoría de TI            43
De 51 a 200 empleados · Dotación y selección de personal         41
Más de 10.001 empleados · Hospitales y atención sanitaria        34
De 11 a 50 empleados · Dotación y selección de personal          33
De 1.001 a 5.000 empleados · Servicios y consultoría de TI       32
                                                                 ..
De 5.001 a 10.000 empleados · Generación de energía eléctrica     1
De 201 a 500 empleados · Telecomunicaciones                       1
De 51 a 200 empleados · Publicaciones en línea                    1
De 5.001 a 10.000 empleados · Bienes inmuebles                    1
De 11 a 50 empleados · Noticias en línea                          1
Name: count, Length: 222, dtype: int64

In [2785]:
linkedin['company_size'] = linkedin['company_size'].apply(lambda row: row.split('·')[0].strip())
linkedin['company_size'].value_counts()

company_size
Más de 10.001 empleados                                251
De 51 a 200 empleados                                  162
De 1.001 a 5.000 empleados                             159
De 11 a 50 empleados                                    92
De 201 a 500 empleados                                  78
De 501 a 1.000 empleados                                76
De 5.001 a 10.000 empleados                             55
Entre 1 y 10 empleados                                  44
Nagesh Kumar Gona ✈ busca personal para este empleo      1
Fabricación de productos de plástico                     1
Name: count, dtype: int64

In [2786]:
def filter_employee(row):
    pattern1 = r"(Más de \d+\.?\d* empleados)"
    pattern2 = r"(De \d+\.?\d* a \d+\.?\d* empleados)"
    pattern3 = r"(Entre \d+\.?\d* y \d+\.?\d* empleados)"

    match1 = re.search(pattern1, str(row))
    match2 = re.search(pattern2, str(row))
    match3 = re.search(pattern3, str(row))

    if match1:
        return match1.group(1)
    elif match2:
        return match2.group(1)
    elif match3:
        return match3.group(1)
    else:
        return np.nan

def clean_employee(row):
    pattern1 = r"Más de (\d+\.?\d*) empleados"
    pattern2 = r"De (\d+\.?\d*) a (\d+\.?\d*) empleados"
    pattern3 = r"Entre (\d+\.?\d*) y (\d+\.?\d*) empleados"

    match1 = re.match(pattern1, str(row))
    match2 = re.match(pattern2, str(row))
    match3 = re.match(pattern3, str(row))

    if match1:
        return match1.group(1)
    elif match2:
        return match2.group(2)
    elif match3:
        return match3.group(2)
    else:
        return np.nan

linkedin['company_size'] = linkedin['company_size'].apply(filter_employee)
linkedin['employees'] = linkedin['company_size'].apply(clean_employee)
linkedin['employees'].value_counts()

employees
10.001    251
200       162
5.000     159
50         92
500        78
1.000      76
10.000     55
10         44
Name: count, dtype: int64

In [2787]:
def standarize_company_size(row):
    if pd.notnull(row):
        if int(row.replace('.', '')) < 50:
            return 'S'
        elif int(row.replace('.', '')) <= 250:
            return 'M'
        elif int(row.replace('.', '')) > 250:
            return 'L'
        else:
            return np.nan

linkedin ['comp_size'] = linkedin['employees']
linkedin['employees'] = linkedin['company_size']
linkedin['company_size'] = linkedin['comp_size']
linkedin['company_size'] = linkedin['company_size'].apply(standarize_company_size)
linkedin = linkedin.drop(columns=['comp_size'])
linkedin['company_size'].value_counts()

company_size
L    619
M    254
S     44
Name: count, dtype: int64

#### Cleaning 'salary_range' column

In [2788]:
linkedin['salary_range'].unique()

array(['90.000 $/año - 120.000 $/año', '100.000 $/año - 150.000 $/año',
       '283.780 $/año - 331.640 $/año', '160.000 $/año - 200.000 $/año',
       nan, '65.000 $/año - 185.000 $/año',
       '145.000 $/año - 180.000 $/año', '95.000 $/año - 130.000 $/año',
       '57.000 $/año - 85.000 $/año', '84.899 $/año - 172.663 $/año',
       '167.200 $/año - 250.800 $/año', '17,87 $/h - 23,96 $/h',
       '200.000 $/año - 240.000 $/año', '136.038 $/año - 182.490 $/año',
       '70.000 $/yr', '200.000 $/año - 300.000 $/año',
       '120.000 $/año - 220.000 $/año', '60.000 $/año - 90.000 $/año',
       '95.301 $/año - 138.186 $/año', '125.000 $/año - 185.000 $/año',
       '144.000 $/año - 270.250 $/año', '150.000 $/año - 200.000 $/año',
       '83.890 $/año - 142.610 $/año', '120.000 $/año - 180.000 $/año',
       '170.112 $/año - 237.000 $/año', '170.000 $/año - 190.000 $/año',
       '20 $/hr', '145.000 $/año - 195.000 $/año', '25 $/h - 30 $/h',
       '67.733 $/año - 91.440 $/año', '125.00

In [2789]:
hour_pattern = r"(\d+) \$\/hr"
year_pattern = r"(\d+\.?\d*) \$\/yr"
hour_range_pattern = r"(\d+\,?\d*) \$\/h - (\d+\,?\d*) \$\/h"
year_range_pattern = r"(\d+\.?\d*) \$\/año - (\d+\.?\d*) \$\/año"
month_range_pattern = r"(\d+\.?\d*) \$\/mes - (\d+\.?\d*) \$\/mes"

In [2790]:
def clean_salary(row, type=None):
    match_h = re.match(hour_pattern, str(row))
    match_y = re.match(year_pattern, str(row))
    match_rh = re.match(hour_range_pattern, str(row))
    match_ry = re.match(year_range_pattern, str(row))
    match_rm = re.match(month_range_pattern, str(row))
    if match_h:
        num_h = match_h.group(1)
        # if type == 'max':
        #     num_h = int(num_h) + 5
        # elif type == 'min':
        #     num_h = int(num_h) - 5
        num_w = int(num_h) * 40
        return int(num_w * 48)
    elif match_y:
        sal = int(match_y.group(1).replace('.', ''))
        # if type == 'max':
        #     sal += 2000
        # elif type == 'min':
        #     sal -= 2000
        return sal
    elif match_rh:
        min_ = match_rh.group(1)
        max_ = match_rh.group(2)
        num_h = (float(min_.replace(',', '.')) + float(max_.replace(',', '.'))) / 2
        # if type == 'max':
        #     num_h = float(max_.replace(',', '.'))
        # elif type == 'min':
        #     num_h = float(min_.replace(',', '.'))
        num_w = num_h * 40
        return int(num_w * 48)
    elif match_ry:
        min_ = match_ry.group(1)
        max_ = match_ry.group(2)
        if '.' not in max_:
            max_ += '000'
        if '.' not in min_:
            min_ += '000'
        num_y = (float(min_.replace('.', '')) + float(max_.replace('.', ''))) / 2
        # if type == 'max':
        #     num_y = float(max_.replace('.', ''))
        # elif type == 'min':
        #     num_y = float(min_.replace('.', ''))
        return int(num_y)
    elif match_rm:
        min_ = match_rm.group(1)
        max_ = match_rm.group(2)
        num_m = (float(min_.replace('.', '')) + float(max_.replace('.', ''))) / 2
        # if type == 'max':
        #     num_m = float(max_.replace('.', ''))
        # elif type == 'min':
        #     num_m = float(min_.replace('.', ''))
        return int(num_m * 12)
    else:
        return np.nan

# linkedin['min_salary'] = linkedin['salary_range'].apply(lambda row: clean_salary(row, 'min'))
linkedin['salary'] = linkedin['salary_range'].apply(clean_salary)
# linkedin['max_salary'] = linkedin['salary_range'].apply(lambda row: clean_salary(row, 'max'))

In [2791]:
linkedin['salary'].unique()

array([105000., 125000., 307710., 180000.,     nan, 162500., 112500.,
        71000., 128781., 209000.,  40156., 220000., 159264.,  70000.,
       250000., 170000.,  75000., 116743., 155000., 207125., 175000.,
       113250., 150000., 203556.,  38400.,  52800.,  79586., 160000.,
       141000., 107500., 174720., 156000., 140000., 106000., 108480.,
       130000., 142500.,  86400., 125250., 108750.,  93250., 153600.,
        96000., 200000.,  79680., 122500., 225000., 172800., 280000.,
       110000., 122880.,  77500.,  54720., 187250.,  80000., 182400.,
       100000., 550000., 152500., 121750., 151750., 103000., 102720.,
       124800.,  62500.,  66240.,  97500., 137000., 275000., 148800.,
        72500., 110400., 400000.,  95300.,  49920., 138900., 214500.,
        55440., 118080.,  99000., 300000., 145000., 105600., 196500.,
       181440., 100800.,  33600.,  11145.,  87360., 122595., 149500.,
       152625., 135000., 129600.,  74668., 138500.,  41280., 106560.,
        60480., 1200

In [2792]:
linkedin.sample(5)

,job_id,job_title,company_name,company_state,salary_range,remote_ratio,employment_type,experience_level,company_size,image_link,employees,salary
424,3766891845,Data Analyst,Ad hoc Research,MD,NaN,NaN,Prácticas,Prácticas,M,https://media.licdn.com/dms/image/C4E0BAQGOHBF...,De 51 a 200 empleados,NaN
226,3758102716,Principal Machine Learning Engineer,Storm3,USA,150.000 $/año - 300.000 $/año,Híbrido,Jornada completa,Intermedio,M,https://media.licdn.com/dms/image/C4D0BAQEGZJV...,De 51 a 200 empleados,225000.0
374,3765449491,Associate Data Scientist,"InVitro Cell Research, LLC",NY,60.000 $/año - 95.000 $/año,Presencial,Jornada completa,Sin experiencia,M,https://media.licdn.com/dms/image/C4E0BAQHN393...,De 11 a 50 empleados,77500.0
380,3765831773,Remote Cruise Planner - Entry Level,Travel by Tilly,AZ,NaN,En remoto,Media jornada,Sin experiencia,S,https://media.licdn.com/dms/image/C4E0BAQHhOeg...,Entre 1 y 10 empleados,NaN
1,3313895534,Signal Processing AI/Machine Learning Engineer,CyberCoders,NC,100.000 $/año - 150.000 $/año,Presencial,Jornada completa,Intermedio,L,https://media.licdn.com/dms/image/C560BAQF7t9X...,De 201 a 500 empleados,125000.0


#### Cleaning 'job_title' column

In [2793]:
linkedin['job_title'].value_counts()

job_title
Machine Learning Engineer                                                           90
Data Engineer                                                                       65
Data Analyst                                                                        64
Data Scientist                                                                      44
Senior Data Scientist                                                               28
                                                                                    ..
Machine Learning Engineer (E-commerce Feed Recommendation) - 2024 Start (MS/PhD)     1
AI Research Engineer                                                                 1
Machine Learning Software Engineer                                                   1
Data Engineer with Mortgage Exp_Remote (Only W2 Contract)                            1
Junior Machine Learning Engineer                                                     1
Name: count, Length: 488, dtype: 

In [2794]:
def standarize_job(row):
    title = str(row).lower()
    if 'data engineer' in title:
        return 'Data Engineer'
    elif 'data analyst' in title:
        return 'Data Analyst'
    elif 'data scientist' in title:
        return 'Data Scientist'
    elif 'data' in title and 'junior' not in title:
        if 'engineer' in title:
            return 'Data Engineer'
        elif 'analyst' in title:
            return 'Data Analyst'
    elif ('machine' in title and 'learning' in title) or ('ml' in title and 'engineer' in title):
        return 'Machine Learning Engineer'
    elif 'deep' in title and 'learning' in title and 'engineer' in title:
        return 'Deep Learning Engineer'
    elif 'junior' in title:
        if 'data' in title and 'engineer' in title:
            return 'Data Engineer'
        elif 'data' in title and 'analyst' in title:
            return 'Data Analyst'
    elif 'business' in title:
        if 'intelligence' in title and 'analyst' in title:
            return 'BI Analyst'
        elif 'analyst' in title:
            return 'Business Analyst'
        else:
            return 'BI Engineer'
    elif 'artificial intelligence' in title or 'ai' in title:
        if 'research engineer' in title:
            return 'Research Engineer'
        elif 'ml' in title:
            return 'Machine Learning Engineer'
        elif 'engineer' in title:
            return 'AI Engineer'
        else:
            return title
    else:
        return str(row)


linkedin['original_title'] = linkedin['job_title']
linkedin['job_title'] = linkedin['job_title'].apply(standarize_job)
linkedin['job_title'].unique()

array(['Machine Learning Engineer', 'CYBER INTEL ANALYST - PEN TESTER',
       'Deep Learning Engineer',
       'Senior Algorithms Engineer (Image Processing)', 'Data Scientist',
       'Data Engineer', 'Data Analyst', 'Deep Learning Researcher',
       'Computer Vision Engineer (Multimodal with Large Language Models)',
       None, 'BI Analyst', 'Artificial Learning Engineer',
       'Research Engineer', 'Computer Vision Engineer',
       'RF Computational Engineer 3', 'Project Scheduler',
       'Adaptive Optics Senior Software Engineer',
       'Finance Analyst (Benelux)', 'Deep Learning Research Intern',
       'Analytics Engineer', 'Applied Scientist',
       'Quantitative Researcher', 'Analyst, Analytics',
       'Computer Vision and Optimization Engineer', 'AI Engineer',
       'Software Company DevOps Engineer', 'BI Engineer',
       'summer intern - r&d/artificial intelligence',
       'Remote Cruise Planner - Entry Level',
       'Remote Travel Consultant - Entry Level',
    

    Categories for 'job_title' column

In [2795]:
categories = [
    'Data Engineer',
    'Data Analyst',
    'Data Scientist',
    'Machine Learning Engineer',
    'Deep Learning Engineer',
    'BI Analyst',
    'Business Analyst',
    'BI Engineer',
    'Research Engineer',
    'AI Engineer'
]

In [2796]:
linkedin.shape

(919, 13)

In [2797]:
linkedin = linkedin[linkedin['job_title'].isin(categories)]

In [2798]:
linkedin.shape
linkedin.sample(3)

,job_id,job_title,company_name,company_state,salary_range,remote_ratio,employment_type,experience_level,company_size,image_link,employees,salary,original_title
246,3759374206,Machine Learning Engineer,Adobe,CA,150.000 $/año - 200.000 $/año,Híbrido,Jornada completa,Intermedio,L,https://media.licdn.com/dms/image/C560BAQFrtK-...,Más de 10.001 empleados,175000.0,2024 AI/ML Intern - Machine Learning Engineer
868,3776946154,Data Engineer,hackajob,USA,100.000 $/año - 140.000 $/año,En remoto,Jornada completa,Intermedio,M,https://media.licdn.com/dms/image/D560BAQF6HYG...,De 51 a 200 empleados,120000.0,Data Engineer
812,3776414165,Data Scientist,Grammarly,USA,181.000 $/año - 262.000 $/año,Híbrido,Jornada completa,Intermedio,L,https://media.licdn.com/dms/image/C560BAQFroT1...,De 501 a 1.000 empleados,221500.0,"Senior Data Scientist, SEO"


In [2802]:
linkedin = linkedin[linkedin['employment_type'] != 'Temporal']
linkedin = linkedin[~linkedin.drop(['salary', 'salary_range'], axis=1).isna().all(axis=1)]
linkedin = linkedin[linkedin['remote_ratio'].notna()]
linkedin = linkedin[linkedin['experience_level'].notna()]
linkedin = linkedin[linkedin['company_size'].notna()]
linkedin.isna().sum()
# linkedin.shape

job_id               0
job_title            0
company_name         0
company_state        0
salary_range        65
remote_ratio         0
employment_type      0
experience_level     0
company_size         0
image_link           0
employees            0
salary              69
original_title       0
dtype: int64

In [2800]:
linkedin.to_csv('../data/linkedin_standarized.csv', index=False)